Goal: realized I didnt save enough station metadata in "find_all_stations", so this is a second pass for more info from the subset previously identified

Order of operations:
- load all_relevant_stations.pkl
- sort by chl|turb|sedi|par|phot|secchi|ssc
- sort those by buoy/glider/ship

In [1]:
%pip install --quiet erddapy

Note: you may need to restart the kernel to use updated packages.


In [7]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import urllib.request
import json
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [ ]:
# load previous search output
FF = pd.read_pickle('all_relevant_stations.pkl')

lib = {'buoy','glider','ship','turbid','ssc','photo','chl','radiation'}

FF["buoy"] = False
FF["glider"] = False
FF["ship"] = False
FF["turbid"] = False
FF["ssc"] = False
FF["photo"] = False
FF["chl"] = False
FF['radiation'] = False
FF['bad'] = False

for jj in range(len(FF)):
    if jj % 100 ==0:
        print(jj)# give a printout every 100 for my sanity

    # make the info URL for this site for this time range
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv"
              )
    e.dataset_id = FF['sites'][jj]
    # e.constraints = {"time>=": min_time, "time<=": max_time}
    info_url = e.get_info_url()
    try:
        ds_info = pd.read_csv(info_url)# make a dataframe for all the metadata for this station
    except:
        FF.loc[jj,"bad"] = True
    else:
        for kwd in lib:
            idx = ds_info["Value"].str.contains(kwd, flags = re.IGNORECASE)
            if any(idx==True)==True:
                FF.loc[jj,kwd] = True

FF = FF[FF.bad == False] 
FF.reset_index(drop=True, inplace=True)

FF = FF.drop(columns="bad")
FF.to_pickle("labeled_relevant_stations.pkl")


0


In [ ]:
FF[(FF.buoy == True) & (FF.radiation == True)]